In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();

  Activating project at `~/Documents/eth_courses/notebooks/optimization/julia`


In [147]:
using PyCall

cp = pyimport("cvxpy");

In [149]:
# DGP requires Variables to be declared positive via `pos=true`.
x = cp.Variable(pos=true)
y = cp.Variable(pos=true)
z = cp.Variable(pos=true)

constrain_1(x,y,z) = 4 * x * y * z + 2 * x * z 
constrain_2(x,y,z) = x
constrain_3(x,y,z) = y 
constrain_4(x,y,z) = z

constrain_4 (generic function with 1 method)

In [150]:
constraints = [
  constrain_1(x,y,z) <= 10, constrain_2(x,y,z) <= 2*y, constrain_3(x,y,z) <= 2*x, constrain_4(x,y,z) >= 1]

4-element Vector{PyObject}:
 PyObject Inequality(Expression(LOG-LOG CONVEX, NONNEGATIVE, ()))
 PyObject Inequality(Variable((), pos=True))
 PyObject Inequality(Variable((), pos=True))
 PyObject Inequality(Constant(CONSTANT, NONNEGATIVE, ()))

In [151]:
objective_fn = x * y * z

PyObject Expression(LOG-LOG AFFINE, NONNEGATIVE, ())

In [154]:
problem = cp.Problem(cp.Maximize(objective_fn), constraints)
problem.solve(gp=true)
print("Optimal value=: ", problem.value)
print("x: ", x.value)
print("y: ", y.value)
print("z: ", z.value)

Optimal value=: 1.9999999938309496x: 0.9999999989682057y: 1.999999974180587z: 1.0000000108569758

# Wing design 

In [50]:
lift_eqn(ρ, V, C_l, S) = 0.5 * ρ * V^2 * C_l * S

let 
ρ = 1.293
V = 10
C_l = 9.5
S = 12

lift_eqn(ρ, V, C_l, S)
    
end


7370.099999999999

In [203]:
# Constants
k = cp.Parameter(pos=true, value= 1.2)
e = cp.Parameter(pos=true, value= 0.95)
mu = cp.Parameter(pos=true, value= 1.78e-5)
rho = cp.Parameter(pos=true, value= 1.23)
tau = cp.Parameter(pos=true, value= 0.12)
N_ult = cp.Parameter(pos=true, value= 3.8)
V_min = cp.Parameter(pos=true, value= 22)
C_Lmax = cp.Parameter(pos=true, value= 1.5)
S_wetratio = cp.Parameter(pos=true, value=  2.05)
W_W_coeff1 = cp.Parameter(pos=true, value= 8.71e-5)
W_W_coeff2 = cp.Parameter(pos=true, value= 45.24)
CDA0 = cp.Parameter(pos=true, value= 0.031)
W_0 = cp.Parameter(pos=true, value=  4940.0)


PyObject Parameter((), pos=True)

In [204]:
# Free Variables

D = cp.Variable(pos=true, name="D")
A = cp.Variable(pos=true, name="A")
S = cp.Variable(pos=true, name="S")
V = cp.Variable(pos=true, name="V")
W = cp.Variable(pos=true, name="W")
Re = cp.Variable(pos=true, name="Re")
C_D = cp.Variable(pos=true, name="C_D")
C_L = cp.Variable(pos=true, name="C_L")
C_f = cp.Variable(pos=true, name="C_f")
W_w = cp.Variable(pos=true, name="W_w")

PyObject Variable((), pos=True)

In [205]:
# Drag model
C_D_fuse = CDA0/S
C_D_wpar = k * C_f * S_wetratio
C_D_ind = (C_L^2)/(pi*A*e)

PyObject Expression(LOG-LOG AFFINE, NONNEGATIVE, ())

In [206]:
# Wing weight model
W_w_strc = W_W_coeff1*(N_ult * A^1.5 * (W_0*W*S)^0.5)/tau
W_w_surf = W_W_coeff2 * S

PyObject Expression(AFFINE, NONNEGATIVE, ())

### Constrains

In [207]:
constraints = [
    C_D >= C_D_fuse + C_D_wpar + C_D_ind,
    W_w >= W_w_surf + W_w_strc,
    D >= 0.5*rho*S*C_D*V^2,
    Re == (rho/mu)*V*(S/A)^0.5,
    C_f == 0.074/Re^0.2,
    W <= 0.5*rho*S*C_L*V^2,
    W <= 0.5*rho*S*C_Lmax*V_min^2,
    W >= W_0 + W_w]

8-element Vector{PyObject}:
 PyObject Inequality(Expression(LOG-LOG CONVEX, NONNEGATIVE, ()))
 PyObject Inequality(Expression(LOG-LOG CONVEX, NONNEGATIVE, ()))
 PyObject Inequality(Expression(LOG-LOG AFFINE, NONNEGATIVE, ()))
 PyObject Equality(Variable((), pos=True), Expression(LOG-LOG AFFINE, NONNEGATIVE, ()))
 PyObject Equality(Variable((), pos=True), Expression(LOG-LOG AFFINE, NONNEGATIVE, ()))
 PyObject Inequality(Variable((), pos=True))
 PyObject Inequality(Variable((), pos=True))
 PyObject Inequality(Expression(AFFINE, NONNEGATIVE, ()))

# Objective

In [208]:
# Minimize the total drag 
objective_fn = D

PyObject Variable((), pos=True)

In [209]:
problem = cp.Problem(cp.Minimize(objective_fn), constraints)

print(problem)

PyObject Problem(Minimize(Variable((), pos=True)), [Inequality(Expression(LOG-LOG CONVEX, NONNEGATIVE, ())), Inequality(Expression(LOG-LOG CONVEX, NONNEGATIVE, ())), Inequality(Expression(LOG-LOG AFFINE, NONNEGATIVE, ())), Equality(Variable((), pos=True), Expression(LOG-LOG AFFINE, NONNEGATIVE, ())), Equality(Variable((), pos=True), Expression(LOG-LOG AFFINE, NONNEGATIVE, ())), Inequality(Variable((), pos=True)), Inequality(Variable((), pos=True)), Inequality(Expression(AFFINE, NONNEGATIVE, ()))])

In [210]:
problem.solve(gp=true)

println("Optimal value=: ", problem.value )

Optimal value=: 303.07477088975094
